In [ ]:
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q torch-cluster -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

     |████████████████████████████████| 7.9 MB 5.9 MB/s 
     |████████████████████████████████| 3.5 MB 4.5 MB/s 
     |████████████████████████████████| 2.5 MB 3.9 MB/s 


# Loading required libraries

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import random
import numpy as np
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
def set_seed(seed=1):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)

set_seed()

# Download dataset from Github

In [ ]:
!wget https://raw.githubusercontent.com/giordamaug/BIONETdatasets/main/CSV/integratedcrispr/edges_integrated2.csv
!wget https://raw.githubusercontent.com/giordamaug/BIONETdatasets/main/CSV/integratedcrispr/nodes_integrated2_update_attr_label_avana0_wang_crispr.csv

--2022-04-08 08:57:49--  https://raw.githubusercontent.com/giordamaug/BIONETdatasets/main/CSV/integratedcrispr/edges_integrated2.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 39025931 (37M) [text/plain]
Saving to: ‘edges_integrated2.csv.1’

edges_integrated2.c 100%[===================>]  37.22M   161MB/s    in 0.2s    

2022-04-08 08:57:50 (161 MB/s) - ‘edges_integrated2.csv.1’ saved [39025931/39025931]

--2022-04-08 08:57:50--  https://raw.githubusercontent.com/giordamaug/BIONETdatasets/main/CSV/integratedcrispr/nodes_integrated2_update_attr_label_avana0_wang_crispr.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent

## To iterate your experiments, select your choice and run from thic cell on...

# Load the attributes and labels from CSV

All node/gene attributes and labels are in one file. First we devide attributes (numeric) from labels (non-numeric) columns.



In [ ]:
import pandas as pd
from sklearn import preprocessing
node_name, label_name = 'name', 'label_CS_ACH_most_freq'
# read node attributes
df = pd.read_csv('/content/nodes_integrated2_update_attr_label_avana0_wang_crispr.csv', 
                 sep='\t', index_col=node_name)   # load csv and set 'name' as index
x = df.select_dtypes(include=np.number)           # Attributes are numeric columns
labels = df.select_dtypes(exclude=np.number)      # labels are not numeric columns

# Set the label
The label is one column or a grouping of values from one label column. In the paper we considered the ten values of `label_CS_ACH_most_freq`, and we grouped as following:
*   `CS0` = `E`ssential class
*   `CS6`-`CS9` = `NE` non essential class

The new label name is `CS0_vs_CS6-9`





In [ ]:
E_class, NE_class = ['CS0'], ['CS6', 'CS7', 'CS8', 'CS9']
new_label_name = 'CS0_vs_CS6-9'
labels[new_label_name] = labels.apply(lambda row: 'E' if row[label_name] in E_class \
                                      else 'NE' if row[label_name] in NE_class \
                                      else row[label_name], axis=1)
labels = labels[labels[new_label_name].isin(['E', 'NE']) == True]       # drop any row contaning NaN or SC1-SC5 as value
genes = labels.index.values
print(f'Selected {len(genes)} genes')

Selected 3814 genes


# Encode the label

In [ ]:
from sklearn import preprocessing
from collections import Counter
encoder = preprocessing.LabelEncoder()
y = encoder.fit_transform(labels[new_label_name].values)  
classes_mapping = dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))
print(classes_mapping, Counter(y))

{'E': 0, 'NE': 1} Counter({1: 3069, 0: 745})


# Load the PPI network
The PPI networks is loaded from a CSV file, where
*   `A` is the column name for edge source (gene name)
*   `B` is the column name for edge target (gene name)
*   `weight` is the column name for edge weight
Only some method use the PPI netoworks, as an example all GCN methods, and Node2Vec.



In [ ]:
ppi = pd.read_csv('/content/edges_integrated2.csv', sep='\t')              # read ppi from CSV file
ppi = ppi.loc[((ppi['A'].isin(genes)) & (ppi['B'].isin(genes)))]           # reduce PPI only to selected nodes/genes
idxlbl = labels.reset_index(drop=True)
idxlbl[node_name] = labels.index
map_gene_to_idx = { v[node_name]: i  for i,v in idxlbl.to_dict('Index').items() }
vfunc = np.vectorize(lambda t: map_gene_to_idx[t])
edges_index = torch.from_numpy(vfunc(ppi[['A','B']].to_numpy().T)) 

## Normalize edge weights

In [ ]:
#@title Normalization edges { form-width: "30%" }
normalize_edge = "minmax" #@param ["", "zscore", "minmax"]
if normalize_edge == 'minmax':
    maximum = ppi.loc[ppi['weight'] != np.inf, 'weight'].max()   # get max other than infinity
    minimum = ppi.loc[ppi['weight'] != np.nan, 'weight'].min()   # get min other than NaN
    ppi['weight'].replace(np.inf,maximum,inplace=True)             # replace ininity with max
    ppi['weight'].replace(np.nan,minimum,inplace=True)             # replace NaN with min
    ppi['weight'] = (ppi['weight'] - minimum) / (maximum - minimum)
elif normalize_edge == 'zscore':
    ppi['weight'] = (ppi['weight'] - ppi['weight'].mean()) / ppi['weight'].std()    

# Select attributes to be used
We identified three sets of attributes:
1. bio attributes, related to gene information (such as, expression, etc.)
2. net attributes, derived from role of gene/node in the network (such as, degree, centrality, etc.)
3. GTEX-* attribute, additional biological information of genes 

In this code snippets the sets of attibutes are defines, and you may choose to subtract some of them from the matrix of node attributes 


In [ ]:
#@title Choose attributes { form-width: "20%" }
import re
bio_attr = False #@param {type:"boolean"}
net_attr = True #@param {type:"boolean"}
gtex_attr = False #@param {type:"boolean"}
bio_attributes = ['gc_content', 'Gtex_kidney', 'gene_disease_ass_count', 'oncodb_expression','orth_count', 'gene_length', \
       'HPA_kidney', 'mf_coal', 'bp_coal', 'cc_coal', 'biogrid_coal', 'kegg_coal', 'reactome_coal', 'ucsc_tfbs_coal', \
       'up_tissue_coal', 'transcript_count']  if bio_attr else []
net_attributes = ['degree', 'ecc', 'clos', 'betw', 'eigen', 'hub', 'trans', 'PR', 'triangles_numb', 'motif1', \
        'motif2', 'motif3', 'motif5', 'strength'] if net_attr else []
r = re.compile('^GTEX*')
gtex_attributes = list(filter(r.match, x.columns)) if gtex_attr else []
x = x.filter(items=bio_attributes+gtex_attributes+net_attributes)

## Normalize attributes
In this snippet of code the matrix of node attributes is corrected by filling NaN with themean in the columns, while Infinte value with maximum. 

The attribute matrix is also reduce by removing all rows correspondning to node not considered, i.e. deleted becaus no label was associated to them.

In [ ]:
#@title Normalization nodes { form-width: "30%" }
normalize_node = "zscore" #@param ["", "zscore", "minmax"]
print(f'Fixing NaN and infinity in X matrix...', end='')
print(f'Found {x.isnull().sum().sum()} NaN values and {np.isinf(x).values.sum()} Inf values')
highest_non_inf = x.max().loc[lambda v: v<np.Inf].max()    # fix infinity (replace with max)
x.replace(np.Inf, highest_non_inf)
for col in x.columns[x.isna().any()].tolist():
  mean_value=x[col].mean()          # Replace NaNs in column with the  mean of values in the same column
  if mean_value is not np.nan:
    x[col].fillna(value=mean_value, inplace=True)
  else:
    x = x.drop(col, 1)

if normalize_node == 'minmax':
  print("X attributes normalization (minmax)...")
  x = (x-x.min())/(x.max()-x.min())
elif normalize_node == 'zscore':
  print("X attributes normalization (zscore)...")
  x = (x-x.mean())/x.std()
x = x.loc[genes]
print(f'New attribute matrix x{x.shape}')

Fixing NaN and infinity in X matrix...Found 214 NaN values and 0 Inf values
X attributes normalization (zscore)...
New attribute matrix x(3814, 14)


# Build PyG storage for network

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import random
from tqdm import tqdm
import sys
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from sklearn import metrics
from sklearn import preprocessing
from collections import Counter

import seaborn as sns

import torch
import torch.nn.functional as F

import torch.optim as optim
from torch_geometric.data import Data
from torch_geometric.transforms import RandomNodeSplit
import torch.nn.functional as F
encoder = preprocessing.LabelEncoder()
y = encoder.fit_transform(labels[new_label_name].values)
classes_mapping = dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))
data = Data(x=torch.from_numpy(x.to_numpy()).float(), edge_index=edges_index, edge_attr=torch.from_numpy(ppi['weight'].values).float(), y = torch.from_numpy(y))
data.num_classes = len(np.unique(y))
tfs =  RandomNodeSplit()
tfs(data)

train_indices = np.arange(0,len(data.x))
data.train_idx = torch.tensor(train_indices[data.train_mask], dtype=torch.long)
data.val_idx = torch.tensor(train_indices[data.val_mask], dtype=torch.long)
data.test_idx = torch.tensor(train_indices[data.test_mask], dtype=torch.long)

print()
print(data)
print('===========================================================================================================')

# Gather some statistics about the graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of node features: {data.num_features}')
print(f'Number of {classes_mapping} classes: {data.num_classes}')
print(f'Class distritions: {Counter(data.y.numpy())}')
print(f'Number of edges: {data.num_edges}')
print(f'Nodes indices: {train_indices}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Number of training nodes: {data.train_mask.sum()}')
print(f'Training node indices: {data.train_idx}')
print(f'Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has edge weights: {data.edge_attr}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is directed: {data.is_directed()}')


Data(x=[3814, 14], edge_index=[2, 107513], edge_attr=[107513], y=[3814], num_classes=2, train_mask=[3814], val_mask=[3814], test_mask=[3814], train_idx=[2314], val_idx=[500], test_idx=[1000])
Number of nodes: 3814
Number of node features: 14
Number of {'E': 0, 'NE': 1} classes: 2
Class distritions: Counter({1: 3069, 0: 745})
Number of edges: 107513
Nodes indices: [   0    1    2 ... 3811 3812 3813]
Average node degree: 28.19
Number of training nodes: 2314
Training node indices: tensor([   2,    3,    4,  ..., 3809, 3810, 3811])
Training node label rate: 0.61
Has isolated nodes: True
Has edge weights: tensor([0.0413, 0.0338, 0.0395,  ..., 0.0003, 0.0066, 0.0047])
Has self-loops: False
Is directed: True


# The Trainer class

In [ ]:
import copy
from dataclasses import dataclass
import os

import torch
import torch.nn as nn
from torch.optim import Adam, lr_scheduler
from tqdm import tqdm

from sklearn import metrics

@dataclass
class RunConfig:  # default parameters from the paper and official implementation
    learning_rate: float = 0.01
    num_epochs: int = 200
    weight_decay: float = 5e-4
    num_warmup_steps: int = 0
    save_each_epoch: bool = False
    output_dir: str = "."

class Trainer:
    def __init__(self, model):
        self.model = model

    def train(self, features, train_labels, val_labels, edge_index, edge_weights, device, run_config, log=True, eval_train=False):
        self.model = self.model.to(device)
        features = features.to(device)
        train_labels = train_labels.to(device)
        val_labels = val_labels.to(device)
        edge_index = edge_index.to(device)  # edhe list and weight
        edge_weights = edge_weights.to(device)

        optimizer = Adam(self.model.parameters(), lr=run_config.learning_rate, weight_decay=run_config.weight_decay)

        # https://huggingface.co/transformers/_modules/transformers/optimization.html#get_linear_schedule_with_warmup
        def lr_lambda(current_step: int):
            if current_step < run_config.num_warmup_steps:
                return float(current_step) / float(max(1, run_config.num_warmup_steps))
            return max(0.0, float(run_config.num_epochs - current_step) /
                       float(max(1, run_config.num_epochs - run_config.num_warmup_steps)))

        scheduler = lr_scheduler.LambdaLR(optimizer, lr_lambda)

        if log:
            print("Training started:")
            print(f"\tNum Epochs = {run_config.num_epochs}")

        best_loss, best_model_accuracy = float("inf"), 0
        best_model_state_dict = None
        if log: train_iterator = tqdm(range(0, int(run_config.num_epochs)))
        else: train_iterator = range(0, int(run_config.num_epochs))
        train_logs = {'train loss' : [], 'train acc' : [], 'train mcc' : [], 'val loss' : [],  'val acc' : [], 'val mcc' : []}
        for epoch in train_iterator:
            self.model.train()
            outputs = self.model(features, edge_index, edge_weights, train_labels)
            loss = outputs[1]

            self.model.zero_grad()
            loss.backward()
            optimizer.step()
            scheduler.step()

            if eval_train: train_loss, train_accuracy, train_mcc, _ , _ = self.evaluate(features, train_labels, edge_index, edge_weights, device)
            val_loss, val_accuracy, val_mcc, _ , _= self.evaluate(features, val_labels, edge_index, edge_weights, device)
            if log: train_iterator.set_description(f"Training loss = {loss.item():.4f}, " f"val loss = {val_loss:.4f}, val acc = {val_accuracy:.2f}, val mcc = {val_mcc:.2f}")

            save_best_model = val_loss < best_loss
            if save_best_model:
                best_loss = val_loss
                best_model_accuracy = val_accuracy
                best_model_state_dict = copy.deepcopy(self.model.state_dict())
            if save_best_model or run_config.save_each_epoch or epoch + 1 == run_config.num_epochs:
                output_dir = os.path.join(run_config.output_dir, f"Epoch_{epoch + 1}")
                #self.save(output_dir)
            if eval_train:
                train_logs['train loss'].append(train_loss)
                train_logs['train acc'].append(train_accuracy)
                train_logs['train mcc'].append(train_mcc)
            train_logs['val loss'].append(val_loss)
            train_logs['val acc'].append(val_accuracy)
            train_logs['val mcc'].append(val_mcc)
        if log:
            print(f"Best model val CE loss = {best_loss:.4f}, best model val accuracy = {best_model_accuracy:.2f}")
        # reloads the best model state dict, bit hacky :P
        self.model.load_state_dict(best_model_state_dict)
        return train_logs 

    def evaluate(self, features, test_labels, edge_index, edge_weights, device):
        features = features.to(device)
        test_labels = test_labels.to(device)
        edge_index = edge_index.to(device)  # edhe list and weight
        edge_weights = edge_weights.to(device)

        self.model.eval()

        outputs = self.model(features, edge_index, edge_weights, test_labels)
        ce_loss = outputs[1].item()

        ignore_label = nn.CrossEntropyLoss().ignore_index
        predicted_label = torch.max(outputs[0], dim=1).indices[test_labels != ignore_label]
        true_label = test_labels[test_labels != -100]
        accuracy = torch.mean((true_label == predicted_label).type(torch.FloatTensor)).item()
        mcc = metrics.matthews_corrcoef(true_label, predicted_label)
        cm = metrics.confusion_matrix(true_label, predicted_label)
        return ce_loss, accuracy, mcc, cm, predicted_label 

    def save(self, output_dir):
        if not os.path.isdir(output_dir):
            os.makedirs(output_dir)

        model_path = os.path.join(output_dir, "model.pth")
        torch.save(self.model.state_dict(), model_path)

# The model GNN 

In [ ]:
from torch_geometric.nn import GCNConv, GATConv, ChebConv
import torch.nn as nn
from torch_geometric.nn import GraphUNet
from torch_geometric.utils import dropout_adj

use_gdc = True
weights=torch.tensor([round(Counter(data.y.numpy())[classes_mapping['NE']]/len(data.y),2), round(Counter(data.y.numpy())[classes_mapping['E']]/len(data.y),2)])
#weights = None

def reset_weights(m):
  '''
    Try resetting model weights to avoid
    weight leakage.
  '''
  for layer in m.children():
   if hasattr(layer, 'reset_parameters'):
    #print(f'Reset trainable parameters of layer = {layer}')
    layer.reset_parameters()

class ChebNetGCN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_hidden_layers=0, dropout=0.1, residual=False, k=2):
        super(ChebNetGCN, self).__init__()

        self.dropout = dropout
        self.residual = residual

        self.input_conv = ChebConv(input_size, hidden_size, k)
        self.hidden_convs = nn.ModuleList([ChebConv(hidden_size, hidden_size, k) for _ in range(num_hidden_layers)])
        self.output_conv = ChebConv(hidden_size, output_size, k)

    def forward(self, x: torch.Tensor, edge_index: torch.Tensor, edge_weight: torch.Tensor, labels: torch.Tensor = None):
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = F.relu(self.input_conv(x, edge_index, edge_weight))
        for conv in self.hidden_convs:
            if self.residual:
                x = F.relu(conv(x, edge_index, edge_weight)) + x
            else:
                x = F.relu(conv(x, edge_index, edge_weight))
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.output_conv(x, edge_index, edge_weight)

        if labels is None:
            return x

        loss = nn.CrossEntropyLoss()(x, labels)
        return x, loss

class GAT(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout=0.1):
        super(GAT, self).__init__()
        torch.manual_seed(42)

        self.conv1 = GATConv(input_size, hidden_size, heads=hidden_size, edge_dim=1, dropout=0.6)
        # On the Pubmed dataset, use heads=8 in conv2.
        self.conv2 = GATConv(hidden_size * hidden_size, output_size, heads=1, edge_dim=1, concat=False,
                             dropout=0.6)
        self.elu = nn.ELU()
        self.dropout = nn.Dropout(dropout)

    def forward(self, x: torch.Tensor, edge_index: torch.Tensor, edge_weight: torch.Tensor, labels: torch.Tensor = None):
        x = self.dropout(x)
        x = self.conv1(x, edge_index, edge_weight)
        x = self.elu(x)
        x = self.dropout(x)
        x = self.conv2(x, edge_index, edge_weight)
        if labels is None:
            return x

        loss = nn.CrossEntropyLoss(weight=weights)(x, labels)
        return x, loss

class OneLayerGCN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout=0.1):
        super(OneLayerGCN, self).__init__()
        torch.manual_seed(42)
        self.conv = GCNConv(input_size, output_size, cached=True, improved=True, add_self_loops=True, normalize=not use_gdc)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)

    def forward(self, x: torch.Tensor, edge_index: torch.Tensor, edge_weight: torch.Tensor, labels: torch.Tensor = None):
        x = self.dropout(x)
        x = self.conv(x, edge_index, edge_weight)
        if labels is None:
            return x

        loss = nn.CrossEntropyLoss(weight=weights)(x, labels)
        return x, loss

class TwoLayerGCN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout=0.1):
        super(TwoLayerGCN, self).__init__()
        torch.manual_seed(42)
        self.conv1 = GCNConv(input_size, hidden_size, cached=True, improved=True, add_self_loops=True, normalize=not use_gdc)
        self.conv2 = GCNConv(hidden_size, output_size, cached=True, improved=True, add_self_loops=True, normalize=not use_gdc)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)

    def forward(self, x: torch.Tensor, edge_index: torch.Tensor, edge_weight: torch.Tensor, labels: torch.Tensor = None):
        x = self.dropout(x)
        x = self.conv1(x, edge_index, edge_weight)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.conv2(x, edge_index, edge_weight)
        if labels is None:
            return x

        loss = nn.CrossEntropyLoss(weight=weights)(x, labels)
        return x, loss

class ThreeLayerGCN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout=0.1):
        super(ThreeLayerGCN, self).__init__()
        torch.manual_seed(42)
        self.conv1 = GCNConv(input_size, hidden_size, cached=True, improved=True, add_self_loops=True, normalize=not use_gdc)
        self.conv2 = GCNConv(hidden_size, hidden_size, cached=True, improved=True, add_self_loops=True, normalize=not use_gdc)
        self.conv3 = GCNConv(hidden_size, output_size, cached=True, improved=True, add_self_loops=True, normalize=not use_gdc)
        self.relu1 = nn.ReLU()
        self.relu2 = nn.ReLU()
        self.dropout = nn.Dropout(dropout)

    def forward(self, x: torch.Tensor, edge_index: torch.Tensor, edge_weight: torch.Tensor, labels: torch.Tensor = None):
        x = self.dropout(x)
        x = self.conv1(x, edge_index, edge_weight)
        x = self.relu1(x)
        x = self.dropout(x)
        x = self.conv2(x, edge_index, edge_weight)
        x = self.relu2(x)
        x = self.dropout(x)
        x = self.conv3(x, edge_index, edge_weight)
        if labels is None:
            return x

        loss = nn.CrossEntropyLoss(weight=weights)(x, labels)
        return x, loss

class GUNet(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout=0.6):
        super().__init__()
        pool_ratios = [2000 / data.num_nodes, 0.5]
        self.unet = GraphUNet(input_size, hidden_size, output_size,
                              depth=3, pool_ratios=pool_ratios)

    def forward(self, x: torch.Tensor, edge_index: torch.Tensor, edge_weight: torch.Tensor, labels: torch.Tensor = None):
        edge_index, _ = dropout_adj(edge_index, edge_attr=edge_weight, p=0.2,
                                    force_undirected=True,
                                    num_nodes=data.num_nodes,
                                    training=self.training)
        x = F.dropout(data.x, p=0.92, training=self.training)

        x = self.unet(x, edge_index)
        if labels is None:
            return x

        loss = nn.CrossEntropyLoss(weight=weights)(x, labels)
        return x, loss

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
def my_plot_confusion_matrix(cm, target_names, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(target_names))
    plt.xticks(tick_marks, target_names, rotation=45)
    plt.yticks(tick_marks, target_names)
    plt.tight_layout()

    width, height = cm.shape

    for x in range(width):
        for y in range(height):
            plt.annotate(str(cm[x][y]), xy=(y, x), 
                        horizontalalignment='center',
                        verticalalignment='center')
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

def plot_history(history, eval_train=False):
    fig, axs = plt.subplots(1, 3)
    if eval_train: leg=['train','val']
    else: leg=['val']
    fig.set_figheight(7)
    fig.set_figwidth(35)
    axs[0].set_title('Loss')
    if eval_train: axs[0].plot(history['train loss'])
    axs[0].plot(history['val loss'])
    axs[0].set(ylabel='loss')
    axs[1].set_title('Accuracy')
    if eval_train: axs[1].plot(history['train acc'])
    axs[1].plot(history['val acc'])
    axs[1].set(ylabel='accuracy')
    axs[2].set_title('MCC')
    if eval_train: axs[2].plot(history['train mcc'])
    axs[2].plot(history['val mcc'])
    axs[2].set(ylabel='mcc')
    for ax in axs.flat:
        ax.set(xlabel='epochs')
        ax.legend(leg, loc='upper left')
    plt.show()

# One-shot validation

In [ ]:
%%script echo skipping
def set_labels(initial_labels, set_mask, ignore_label):
    initial_labels[~set_mask] = ignore_label
    return initial_labels

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

ignore_index = nn.CrossEntropyLoss().ignore_index  # = -100, used to ignore not allowed labels in CE loss
train_labels = set_labels(data.y.clone(), data.train_mask, ignore_index)
val_labels = set_labels(data.y.clone(), data.val_mask, ignore_index)
test_labels = set_labels(data.y.clone(), data.test_mask, ignore_index)

# training parameters
run_config = RunConfig(learning_rate=0.01, num_epochs=100, weight_decay=5e-4, output_dir="./gcn-pytorch/saved/")

model = ChebNetGCN(input_size=data.x.shape[1],hidden_size=16,output_size=data.num_classes,dropout=0)
#model = GUNet(input_size=data.x.shape[1],hidden_size=16,output_size=data.num_classes,dropout=0)
#model = GAT(input_size=data.x.shape[1],hidden_size=8,output_size=data.num_classes,dropout=0)
print(model)

# training
trainer = Trainer(model)
history = trainer.train(data.x, train_labels, val_labels, data.edge_index, data.edge_attr, device, run_config, log=True)

# evaluating
ce_loss, accuracy, mcc, cm, preds = trainer.evaluate(data.x, test_labels, data.edge_index, data.edge_attr, device)
print(f'\nPerformance on kidney:\n- test accuracy = {accuracy:.3f}\n- test mcc = {mcc:.3f}\n')
my_plot_confusion_matrix(cm, target_names=classes_mapping.keys())


ChebNetGCN(
  (input_conv): ChebConv(14, 16, K=2, normalization=sym)
  (hidden_convs): ModuleList()
  (output_conv): ChebConv(16, 2, K=2, normalization=sym)
)
Training started:
	Num Epochs = 100


  0%|          | 0/100 [00:00<?, ?it/s]


TypeError: ignored

In [ ]:
%%script echo skipping
plot_history(history)

skipping


# k-fold validation

In [ ]:
#@title Choose GNN { form-width: "20%" }
netmodel = "OneLayerGCN" #@param ["OneLayerGCN", "TwoLayerGCN", "ChebNetGCN"]
epochs = 200 #@param {type:"slider", min:10, max:1000, step:10}
def set_labels(initial_labels, set_mask, ignore_label):
    initial_labels[~set_mask] = ignore_label
    return initial_labels
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from sklearn.model_selection import StratifiedKFold, KFold, train_test_split
from sklearn.metrics import *
from tqdm import tqdm
nfolds = 5
kf = KFold(n_splits=nfolds)
accuracies = []
mccs = []
cmaccumulated = np.array([[0,0],[0,0]])
columns_names = ["Accuracy","BA", "Sensitivity", "Specificity","MCC", 'CM']
scores = pd.DataFrame(columns=columns_names)
for fold, (train_index, test_index) in enumerate(tqdm(kf.split(np.arange(data.num_nodes)), total=kf.get_n_splits(), desc=f"{nfolds}-fold")):
    train_labels = y[train_index]
    train_index, val_index = train_test_split(train_index, test_size=0.025, stratify=train_labels)
    tr_index  = torch.tensor(train_index)
    val_index  = torch.tensor(val_index)
    ts_index = torch.tensor(test_index)
    train_mask = torch.BoolTensor([False]*data.num_nodes)
    val_mask = torch.BoolTensor([False]*data.num_nodes)
    test_mask = torch.BoolTensor([False]*data.num_nodes)
    train_mask.scatter_(0, tr_index, True)
    val_mask.scatter_(0, val_index, True)
    test_mask.scatter_(0, ts_index, True)

    ignore_index = nn.CrossEntropyLoss().ignore_index  # = -100, used to ignore not allowed labels in CE loss
    train_labels = set_labels(data.y.clone(), train_mask, ignore_index)
    val_labels = set_labels(data.y.clone(), val_mask, ignore_index)
    test_labels = set_labels(data.y.clone(), test_mask, ignore_index)
    # training parameters
    run_config = RunConfig(learning_rate=0.01, num_epochs=epochs, weight_decay=5e-4)
    model = globals()[netmodel](input_size=data.x.shape[1],hidden_size=16,output_size=data.num_classes,dropout=0)

    # training
    trainer = Trainer(model)
    print(device)
    trainer.train(data.x, train_labels, val_labels, data.edge_index, data.edge_attr, device, run_config, log=False)

    # evaluating
    ce_loss, accuracy, mcc, cm, preds = trainer.evaluate(data.x, test_labels, data.edge_index, data.edge_attr, device)
    accuracies.append(accuracy)
    mccs.append(mcc)
    cmaccumulated += cm
    true_label = test_labels[test_labels != -100]
    scores = scores.append(pd.DataFrame([[accuracy_score(true_label.cpu().numpy(), preds.cpu().numpy()), balanced_accuracy_score(true_label.cpu().numpy(), preds.cpu().numpy()), 
        cm[0,0]/(cm[0,0]+cm[0,1]), cm[1,1]/(cm[1,0]+cm[1,1]), 
        matthews_corrcoef(true_label.cpu().numpy(), preds.cpu().numpy()), cm]], columns=columns_names, index=[fold]))
df_scores = pd.DataFrame(scores.mean(axis=0)).T
df_scores.index=[f'{netmodel}']
df_scores['CM'] = [cmaccumulated]
print(df_scores.to_latex())
df_scores
my_plot_confusion_matrix(cmaccumulated, target_names=classes_mapping.keys())


5-fold:   0%|          | 0/5 [00:00<?, ?it/s]

cuda


RuntimeError: ignored